# Loading Data

In [24]:
import pandas as pd

# Read the data
data  = pd.read_csv("../Data/Combined.csv")

# Split the data into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)
X_train = train['Tweets_clean_more']
y_train = train['Label']
X_test = test['Tweets_clean_more']
y_test = test['Label']

# Tokenization & Padding

In [25]:
from keras.preprocessing.text import Tokenizer

# Tokenize the data
tokenizer_1 = Tokenizer(num_words=5000, oov_token="")
tokenizer_1.fit_on_texts(X_train)
X_train_tokenized = tokenizer_1.texts_to_sequences(X_train)
X_test_tokenized = tokenizer_1.texts_to_sequences(X_test)

In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad the data
maxlen = 100
X_train = pad_sequences(X_train_tokenized, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test_tokenized, padding='post', maxlen=maxlen)

# Embedding Matrix - GloVe

In [27]:
import numpy as np
"""
vocab_size = len(tokenizer.word_index) + 1

embedding_index = dict()
f = open("../Input Source/golve.twitter.27B/glove.twitter.27B.200d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embedding_index))
"""

'\nvocab_size = len(tokenizer.word_index) + 1\n\nembedding_index = dict()\nf = open("../Input Source/golve.twitter.27B/glove.twitter.27B.200d.txt")\nfor line in f:\n    values = line.split()\n    word = values[0]\n    coefs = np.asarray(values[1:], dtype=\'float32\')\n    embedding_index[word] = coefs\nf.close()\nprint(\'Loaded %s word vectors.\' % len(embedding_index))\n'

# BERT

In [28]:
maxlen = 160
lr = 1e-5 #
num_epochs = 3 
batch_size=16

In [29]:
#-wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

!pip install tensorflow_hub

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint


def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [30]:
def build_model(bert_layer, max_len=512, lr=1e-5):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    _, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(clf_output)
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=lr), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [32]:
! pip install bert-tensorflow
from bert import tokenization

#%%time
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=True)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

X_train_Bert = train['Tweets_clean_more']
y_train = train['Label']
X_test_Bert = test['Tweets_clean_more']
y_test = test['Label']

train_input = bert_encode(X_train_Bert, tokenizer, max_len=maxlen)
test_input = bert_encode(X_test_Bert, tokenizer, max_len=maxlen)
train_labels = y_train
test_labels = y_test

bert_model = build_model(bert_layer, max_len=maxlen, lr=lr)
bert_model.summary()

UnparsedFlagAccessError: Trying to access flag --preserve_unused_tokens before flags were parsed.

In [ ]:
checkpoint = ModelCheckpoint('bertmodel.h5', monitor='val_accuracy', save_best_only=True)

bert_history = bert_model.fit(
    train_input, train_labels,
    validation_data=(val_input, val_labels),
    epochs=num_epochs,
    callbacks=[checkpoint], 
    #class_weight=class_weight,
    batch_size=batch_size
)